In [8]:
from pyabc import (ABCSMC, Distribution, RV,
                   MultivariateNormalTransition)
from pyabc.populationstrategy import AdaptivePopulationSize
import matplotlib.pyplot as plt
%matplotlib inline
import os
import tempfile
import pandas as pd
import scipy as sp
import numpy as np
import subprocess
from io import BytesIO
db_path = ("sqlite:///" +
           os.path.join(tempfile.gettempdir(), "test.db"))

In [9]:
def simulate(**pars):
    """Wrapper to simulate the myokit model.
    
    Simulates in a subprocess running python2 by passing
    parameters as arguments to (another) wrapper script.
    
    Args:
        pars: Parameters as kwargs.
        
    Returns:
        Dataframe with simulated output or empty if
        the simulation failed.
    """
    myokit_python = ("/Users/charles/miniconda3/envs" +
                     "/ion_channel_ABC/bin/python")
    script = "run_channel.py"
    args = [myokit_python, script]
    for p in pars:
        try:
            args.append("-" + str(p))
            args.append(str(pars[p]))
        except:
            print("Error: " +
                  "args is " + str(args))
    re = subprocess.run(args, stdout=subprocess.PIPE)
    if len(re.stdout) > 0:
        d = pd.read_table(BytesIO(re.stdout),
                          delim_whitespace=True,
                          header=0, index_col=0)
    else:
        d = pd.DataFrame({})
    return d

In [10]:
myokit_python = ("/Users/charles/miniconda3/envs" +
                 "/ion_channel_ABC/bin/python")
args = [myokit_python, "get_measurements.py"]
re = subprocess.run(args, stdout=subprocess.PIPE)
measurements = pd.read_table(BytesIO(re.stdout),
                             delim_whitespace=True,
                             header=0, index_col=0)

In [11]:
prior = Distribution(
    g_Na=RV("uniform", 0, 100),
    E_Na=RV("uniform", 0, 100),
    p1=RV("uniform", 0, 100),
    p2=RV("uniform", -10, 10),
    p3=RV("uniform", 0, 1),
    p4=RV("uniform", 0, 100),
    p5=RV("uniform", -1, 1),
    p6=RV("uniform", 0, 1),
    p7=RV("uniform", 0, 100),
    q1=RV("uniform", 0, 100),
    q2=RV("uniform", 0, 10))

In [12]:
def distance(sim, obs):
    if sim.empty:
        return float("inf")
    dist = 0
    for i in obs.exp.unique():
        try:
            err = np.sum(np.square(obs[obs.exp == i].y - sim[sim.exp == i].y))
        except:
            return float("inf")
        err = pow(err / len(obs[obs.exp == i].y), 0.5)
        err /= np.ptp(obs[obs.exp == i].y)
        dist += err
    return dist

In [13]:
def simulate_pyabc(parameter):
    res = simulate(**parameter)
    return res

In [16]:
abc = ABCSMC(simulate_pyabc, prior, distance,
             population_size=AdaptivePopulationSize(20, 0.8))
abc_id = abc.new(db_path, measurements)

INFO:Epsilon:initial epsilon is 19.708409871296546
INFO:History:Start <ABCSMC(id=16, start_time=2018-04-20 15:10:16.074299, end_time=None)>


In [ ]:
history = abc.run(max_nr_populations=10, minimum_epsilon=0.8)

INFO:ABC:t:0 eps:19.708409871296546
INFO:ABC:t:1 eps:10.826497065089352
INFO:Adaptation:Change nr particles 20 -> 549
